# 第八章
# 数据规整:连接、联合与重塑

In [46]:
# 引入包
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

## 8.1.分层索引
分层索引时一个重要特性，它允许你在一个轴上拥有多个索引层级，分层索引提供了一种在**更低维度的形式中处理更高维度数据的方式**。

In [47]:
data = Series(np.random.randn(9), index=[list('aaabbccdd'), [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    2.070680
   2   -0.014472
   3    0.998940
b  1   -0.738948
   3   -1.160696
c  1   -0.985052
   2    0.939013
d  2   -0.105066
   3   -0.919354
dtype: float64

In [48]:
data = Series(np.random.randn(9), index=[list('aaabbccdd'), [1, 1, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.263937
   1   -0.154209
   3    1.382462
b  1    1.098556
   3    0.121455
c  1    0.518676
   2    1.286209
d  2    0.372350
   3    0.172819
dtype: float64

In [49]:

data.index

MultiIndex([('a', 1),
            ('a', 1),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [50]:
# 按第一层level取
data['b']

1    1.098556
3    0.121455
dtype: float64

In [51]:
# 切片操作相同
data['b':'c']

b  1    1.098556
   3    0.121455
c  1    0.518676
   2    1.286209
dtype: float64

In [52]:
data.loc[:, 2]

c    1.286209
d    0.372350
dtype: float64

In [53]:
data

a  1   -0.263937
   1   -0.154209
   3    1.382462
b  1    1.098556
   3    0.121455
c  1    0.518676
   2    1.286209
d  2    0.372350
   3    0.172819
dtype: float64

In [54]:
data.unstack()

ValueError: Index contains duplicate entries, cannot reshape

unstack的逆操作是stack

In [45]:
data.unstack().stack()# 可以看出是自动忽略NA的

a  1    0.730333
   2    0.279206
   3   -2.137611
b  1    0.743132
   3   -0.704015
c  1    2.023873
   2   -2.360918
d  2   -1.575139
   3    0.009659
dtype: float64

In [46]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

分层的层级可以有name，如果层级有名称，这些名称会在display输出中显示：

In [47]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [48]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 8.1.1 重排序和层级排序

swaplevel接收两个层级序号或层级名称，返回一个进行了层级变更的新对象，但是数据是不变的

In [49]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

另一方面，sort_index只能在单一层级上对数据进行排序。在进行层级变换时，使用sort_index以使得结果按照层级进行字典排序

In [50]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [51]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 按层级进行汇总统计
通过level可以指定想要的某个特定的轴上进行聚合。

In [52]:
# 这两种写法都可以
frame.sum(level=1)
# frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [53]:
frame = DataFrame({'a':range(7),
                   'b':range(7, 0, -1),
                   'c':['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd':[0,1,2,0,1,2,3]})
frame


,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame的set_index函数会生成一个新的DataFrame，新的DataFrame使用一个或多个列作为索引：

In [54]:
numbers = frame['c'].unique()
for i in numbers:
    display(frame.loc[frame['c'] == i])

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2


,a,b,c,d
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [55]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下，这些列会从DataFrame中删除，也可以通过设置参数留下他们

In [56]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

另一方面，reset_index是set_index的反操作，分层索引的索引层级会被移动到列中：

In [57]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [58]:
data = Series(np.random.randn(9), index=[list('aaabbccdd'), [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.849163
   2   -0.876837
   3    0.662765
b  1   -0.170492
   3    0.916552
c  1    1.804692
   2   -1.105235
d  2    0.315952
   3    2.155167
dtype: float64

In [59]:
data.reset_index()

,level_0,level_1,0
0,a,1,-0.849163
1,a,2,-0.876837
2,a,3,0.662765
3,b,1,-0.170492
4,b,3,0.916552
5,c,1,1.804692
6,c,2,-1.105235
7,d,2,0.315952
8,d,3,2.155167


## 8.2 联合与合并数据集
包含pandas对象的数据可以通过多种方式联合在一起
+ pandas.merge根据一个或多个键将行进行连接，对于SQL或其他关系型数据库的用户来说，这种方式比较熟悉，它实现的是数据库的连接操作
+ pandas.concat使对象在轴向上进行粘合或“堆叠”
+ combine_first实例方法允许将重叠的数据拼接在一起。

### 8.2.1 数据库风格的DataFrame连接

In [60]:
df1 = DataFrame({
    'key':list('bbacaab'),
    'data1':range(7)
})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [61]:
df2 = DataFrame({
    'key':list('abd'),
    'data2':range(3)
})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [62]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [63]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [64]:
df3 = DataFrame({
    'lkey':list('bbacaab'),
    'data1':range(7)
})
df4 = DataFrame({
    'rkey':list('abd'),
    'data2':range(3)
})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


默认情况下，merge做内连接，取交集。可以选择其他选项left，right，outer。分别是左连接，右连接，并集

In [65]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


尽管不是很直观，但*多对多*的合并有明确的行为。下面是一个例子

In [66]:
df1 = DataFrame({
    'key':list('bbacab'),
    'data1':range(6)
})
df2 = DataFrame({
    'key':list('ababd'),
    'data2':range(5)
})
display(df1)
display(df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [67]:
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


多对多的连接是行的笛卡尔积，由于左边的DataFrame中有三个’b‘行，而在右边有两行，因此结果中有6个’b‘行。

In [68]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [69]:
left = DataFrame({
    'key1':['foo', 'foo', 'bar'],
    'key2':['one', 'one', 'one'],
    'lval':[1, 2, 3]
})
right = DataFrame({
    'key1':['foo', 'foo', 'bar', 'bar'],
    'key2':['one', 'one', 'one', 'two'],
    'rval':[4, 5, 6, 7]
})
display(left)
display(right)
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval
0,foo,one,1
1,foo,one,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1.0,4
1,foo,one,1.0,5
2,foo,one,2.0,4
3,foo,one,2.0,5
4,bar,one,3.0,6
5,bar,two,NaN,7


合并操作中最后一个要考虑的问题是如何处理重叠的列名，虽然可以手动解决重叠问题。

但是merge有一个suffixes后缀选项，看以下实例

In [70]:
pd.merge(left, right, on='key1')#结果是key2_x, key2_y

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,one,2,one,4
3,foo,one,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [71]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,one,2,one,4
3,foo,one,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [78]:
pd.merge(left, right, on='key1', how='outer', suffixes=('_left', '_right'), indicator=True)

,key1,key2_left,lval,key2_right,rval,_merge
0,foo,one,1,one,4,both
1,foo,one,1,one,5,both
2,foo,one,2,one,4,both
3,foo,one,2,one,5,both
4,bar,one,3,one,6,both
5,bar,one,3,two,7,both


### merge函数参数
(left: DataFrame, right: DataFrame | Series, how: str = ..., on: Any = ..., left_on: Any = ..., right_on: Any = ..., left_index: bool = ..., right_index: bool = ..., sort: bool = ..., suffixes: Sequence[str | None] = ..., copy: bool = ..., indicator: bool | str = ..., validate: str = ...) -> DataFrame

参数|描述
:--|:--
left|左df
right|右df
how|上边说了
on|需要连接的列名
left_on|左连接键名
right_on|右连接键名
left_index|使用left的行索引作为连接键
right_index|right
sort|通过连接键按字母顺序对合并数据惊醒排列
suffixes|默认_x,_y
copy|如果为False，则默写特殊情况下避免将数据复制到结果数据结构中，默认总是复制
indicator|添加一个特殊列_merge，指示数据来源：’both‘,'right_only', 'left_only'

### 8.2.2 根据索引合并

In [79]:
left1 = DataFrame({
    'key': list('abaabc'),
    'value': range(6)
})
right1 = DataFrame({'group_val': [3.5, 7]},
                    index=['a', 'b'])
display(left1)
display(right1)

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [80]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [81]:
left2 = DataFrame([[1.,2.], [3., 4.],[5., 6.]], index=['a', 'c', 'e'])
right2 = DataFrame([[7., 8.], [9., 10.], [11.,12.], [13., 14.]], index=list('bcde'), columns=['Missouri', 'Alabama'])

In [82]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,0,1,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame有一个方便的join实例方法，用于按照索引合并。该方法也可以用于合并过个索引相同或相似但没有重叠列的DataFrame对象。在之前的例子中，我们可以这样写

In [83]:
left2.join(right2, how='outer')

,0,1,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### 8.2.3 沿轴向连接
另一种数据组合操作可互换地称为拼接、绑定或堆叠。NumPy的concatenate函数可以在numpy上实现该功能

In [84]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [85]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [86]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [87]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [88]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [89]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [90]:
pd.concat([s1, s4], axis=1, join='outer')

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


拼接在一起的各部分无法在结果里显示

In [92]:
s1

a    0
b    1
dtype: int64

In [93]:
s2

c    2
d    3
e    4
dtype: int64

In [94]:
s3

f    5
g    6
dtype: int64

In [91]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [95]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


沿着轴向axis=1连接Series的时候，keys则成为DataFrame的列头。默认为整数列头

In [98]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [100]:
df1 = DataFrame(np.arange(6).reshape(3,2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [101]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果你传递的是对象的字典而不是列表，则字典的键会用于keys选项

In [102]:
pd.concat({
    'level1':df1,
    'level2':df2
}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

还有一些额外的参数负责多层索引生成，我们可以使用names参数命名生成的轴层级：

In [104]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
                      names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [105]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

,a,b,c,d
0,-0.355543,-1.066650,0.922216,-0.032076
1,1.898055,0.070428,0.939886,-0.404450
2,0.511101,-0.353334,0.067727,-0.511237


In [109]:
pd.concat([df1, df2], ignore_index=False)#表示

,a,b,c,d
0,-0.355543,-1.066650,0.922216,-0.032076
1,1.898055,0.070428,0.939886,-0.404450
2,0.511101,-0.353334,0.067727,-0.511237
0,0.234618,0.292350,NaN,-1.267203
1,1.096042,0.464650,NaN,-0.088984


In [111]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.355543,-1.066650,0.922216,-0.032076
1,1.898055,0.070428,0.939886,-0.404450
2,0.511101,-0.353334,0.067727,-0.511237
3,0.234618,0.292350,NaN,-1.267203
4,1.096042,0.464650,NaN,-0.088984


### 8.2.4 联合重叠数据

In [112]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=list('fedcba'))
b = pd.Series([0., np.nan, 2, np.nan, np.nan, 5.], index=list('abcdef'))
display(a)
display(b)

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [113]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

打补丁方法，combine_first方法

In [116]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

## 8.3 重塑和透视
重排列表格型数据有多种基础操作，称为重塑或透视

### 8.3.1 使用多层索引进行重塑

In [117]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


在这份数据上使用stack方法会将列数据透视到行上，产生一个新的Series

In [119]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [120]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


 默认情况下，最内层是已拆堆的，可以传入一个层级序号或名称来拆分一个不同的层级

In [123]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [128]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [131]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=list('cde'))
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [133]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


默认情况下，堆叠会过滤出缺失值。设置`dropna=False`就能不过滤缺失值

In [132]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

当你再DataFrame中拆堆时，被拆堆得层级会变为结果中最低的层级

In [136]:
df = DataFrame({
    'left':result,
    'right':result + 5,
}, columns=pd.Index(['left', 'right'], name='side'))
df#这个例子是有两列值的

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [137]:
df.unstack('state')#这里就是拆成了最低的层级

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [138]:
df.unstack('state').stack('state')

side             left  right
number state                
one    Ohio         0      5
       Colorado     3      8
two    Ohio         1      6
       Colorado     4      9
three  Ohio         2      7
       Colorado     5     10

### 8.3.2 将长透视为宽
在数据库和CSV中储存多时间序列的方式就是所谓的长格式或堆叠格式，让我们载入一些示例数据，然后做少量时间序列规整和其他的数据清洗操作

In [57]:
data = pd.read_csv('pydata-notebook/examples/macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008.0,3.0,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008.0,4.0,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009.0,1.0,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009.0,2.0,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [58]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [59]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [77]:
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')#标记，PeriodIndex.to_tiemstamp
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [87]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

这种数据就是所谓多时间序列的长格式，或称为具有两个或多个键的其他观测数据

此时用pivot来操作，前两个参数分别为行和列的索引，然后是可选的数值作为值填充DataFrame

In [88]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [92]:
ldata['value2'] = np.random.randn(len(ldata))
ldata

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-0.564761
1,1959-03-31 23:59:59.999999999,infl,0.000,1.152341
2,1959-03-31 23:59:59.999999999,unemp,5.800,-1.647981
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.564960
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.690473
...,...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370,-0.937932
605,2009-06-30 23:59:59.999999999,unemp,9.200,0.123837
606,2009-09-30 23:59:59.999999999,realgdp,12990.341,1.460068
607,2009-09-30 23:59:59.999999999,infl,3.560,0.415564


In [94]:
pivoted = ldata.pivot('date', 'item')
pivoted

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8  1.152341 -0.564761   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1 -0.690473  0.564960   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3  0.081609  0.041613   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.996156  2.226642   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -2.271443  0.860553   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  0.178066  0.687308   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.081000 -3.286399   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1 -1.402012 -1.387016   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2 -0.937932 -1.178577   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  0.415564  1.460068   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.647981  
1959-06-30 23:59:59.999999999 -1.332508  
1959-09-30 23:59:59.999999999  0.318999  
1959-12-31 23:59:59.999999999 -0.315557  
1960-03-31 23:59:59.999999999 -0.033399  
...                                 ...  
2008-09-30 23:59:59.999999999  0.354102  
2008-12-31 23:59:59.999999999  0.627008  
2009-03-31 23:59:59.999999999 -0.565021  
2009-06-30 23:59:59.999999999  0.123837  
2009-09-30 23:59:59.999999999  1.364699  

[203 rows x 6 columns]

In [96]:
pivoted.columns#果然是一个多层索引

MultiIndex([( 'value',    'infl'),
            ( 'value', 'realgdp'),
            ( 'value',   'unemp'),
            ('value2',    'infl'),
            ('value2', 'realgdp'),
            ('value2',   'unemp')],
           names=[None, 'item'])

**请注意**：pivot方法等价于使用set_index创建分层索引，然后调用unstack

In [99]:
'''
	date	item	value	value2
0	1959-03-31 23:59:59.999999999	realgdp	2710.349	-0.564761
1	1959-03-31 23:59:59.999999999	infl	0.000	1.152341
2	1959-03-31 23:59:59.999999999	unemp	5.800	-1.647981
3	1959-06-30 23:59:59.999999999	realgdp	2778.801	0.564960
4	1959-06-30 23:59:59.999999999	infl	2.340	-0.690473
...	...	...	...	...
604	2009-06-30 23:59:59.999999999	infl	3.370	-0.937932
605	2009-06-30 23:59:59.999999999	unemp	9.200	0.123837
606	2009-09-30 23:59:59.999999999	realgdp	12990.341	1.460068
607	2009-09-30 23:59:59.999999999	infl	3.560	0.415564
608	2009-09-30 23:59:59.999999999	unemp	9.600	1.364699'''
ldata.set_index(['date', 'item']).unstack()

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8  1.152341 -0.564761   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1 -0.690473  0.564960   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3  0.081609  0.041613   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.996156  2.226642   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -2.271443  0.860553   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  0.178066  0.687308   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.081000 -3.286399   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1 -1.402012 -1.387016   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2 -0.937932 -1.178577   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  0.415564  1.460068   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -1.647981  
1959-06-30 23:59:59.999999999 -1.332508  
1959-09-30 23:59:59.999999999  0.318999  
1959-12-31 23:59:59.999999999 -0.315557  
1960-03-31 23:59:59.999999999 -0.033399  
...                                 ...  
2008-09-30 23:59:59.999999999  0.354102  
2008-12-31 23:59:59.999999999  0.627008  
2009-03-31 23:59:59.999999999 -0.565021  
2009-06-30 23:59:59.999999999  0.123837  
2009-09-30 23:59:59.999999999  1.364699  

[203 rows x 6 columns]

### 8.3.3 将‘宽’透视为‘长’

在DataFrame中，pivot方法的反操作是pandas.melt。与将一列变换为新的DataFrame中的多列不同，它将多列合并成一列：

In [100]:
df = pd.DataFrame({
    'key': ['foo', 'bar', 'baz'],
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [106]:
melted = df.melt('key')
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [107]:
reshaped = melted.pivot('key', 'variable', 'value')#key作为唯一索引，variable作为列索引value作为值
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [108]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7
